In [108]:
import urllib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [109]:
def inner_html(obj):
    # shamelessly stolen
    
    if obj != None:
        string = obj.encode_contents().decode('utf-8').strip()
    else:
        string = ""
    return string

def company_link(object_child):
    # filter out link
    
    c_content = object_child.find_all('a')
    if c_content != None and len(c_content) > 2:
        return c_content[1]["href"]
    else:
        return []
        
def company_desc(object_child):
    # inner html of description 'p' tag

    page_text = object_child.find('p')
    return inner_html(page_text)

def strip_tag(string):
    # return name of company, description
    
    n = string.count('>')/2
#     n = 1
#     print(n)
    result = string.split('>',2*n)
    if len(result) < 2*n + 1:
        return "", string
    else:
        return result[n].split('<')[0], result[2*n]


In [110]:
link = "https://www.eranyc.com/companies/"
page_response = requests.get(link)

content = BeautifulSoup(page_response.content, "html.parser")

In [111]:

results = list()
# item_class="panel__description"
item_class="panel__wysiwyg"
parent = content.find_all('div', class_=item_class)

for i, child in enumerate(parent):
    listing = {}
    
    if child != None:
        a_href = company_link(child)
        desc   = company_desc(child)
        name, desc = strip_tag(desc)
        desc = name + desc
        
        listing['a'] = a_href
        listing['name'] = name
        listing['desc'] = desc
#         listing['full'] = child

        results.append(listing)

#         if i < 10:
#             print(listing)
print(str(len(results)) + " results found. filtering out linkless "
      "(my bad, its easier to discard links that don't work)")

187 results found. filtering out linkless (my bad, its easier to discard links that don't work)


In [112]:
def condition(l):
    return (
        l['name'] != "" and
        type(l['a']) != type([]) and
        l['name'][:20] != l['desc'][:20]
    )

    
easy = [l for l in results if condition(l)]

hard = [l for l in results if not condition(l)]

print(str(len(easy)) + " good")
print(str(len(hard)) + " bad")

print(hard[1])


137 good
50 bad
{'a': u'http://www.agolo.com', 'name': u' helps readers get to the point faster, by applying the world\u2019s most advanced summarization software to news, chat, voice, and video. The company\u2019s AI platform can analyze thousands of documents and produce a summary of key points, specific to the reader\u2019s interests, in real-time. Agolo has the largest dataset of summaries in existence and this powers their AI engine.', 'desc': u' helps readers get to the point faster, by applying the world\u2019s most advanced summarization software to news, chat, voice, and video. The company\u2019s AI platform can analyze thousands of documents and produce a summary of key points, specific to the reader\u2019s interests, in real-time. Agolo has the largest dataset of summaries in existence and this powers their AI engine.In a world overwhelmed by content, Agolo enables efficient consumption so readers can spend more time thinking.'}


In [113]:
print("exporting to csv")

headers = ['name','desc','a']    
rows = [[x['name'].encode('utf-8'),x['desc'].encode('utf-8'),x['a'].encode('utf-8')] for x in easy]

with open('result.csv','w+') as csvfile:
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
      
    # writing the fields 
    csvwriter.writerow(headers) 
      
    # writing the data rows 
    csvwriter.writerows(rows)

exporting to csv
